In [1]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import seaborn as sns
import plotly.express as px

#Carregando as bibliotecas necessárias para a análise estatística
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample

In [2]:
#ponte de conexão com o redshift
connection = psycopg2.connect(dbname="dw_amorsaude",
                  user="",
                  password="",
                  host ="redshift.amazonaws.com",
                  port = 5439)

In [3]:
#query
df_emails = '''select * from stg_pacientes'''

In [4]:
#df = pd.read_sql_query(df_emails,con=connection)
#df

In [8]:
caminho_arquivo = '/home/gabriel/dbt-projects/python_files/analises/files/xls unificados.xlsx'

dados_excel = pd.read_excel(caminho_arquivo,  engine='openpyxl')

,Nome da Origem,Record ID,Nome,Sobrenome,First name,Número de celular,Número de telefone,Número de telefone do WhatsApp,E-mail
0,2 unidade Fortaleza.xlsx,732319301,Raquel,Felix,NaN,NaN,NaN,NaN,raquelfber@gmail.com
1,2 unidade Fortaleza.xlsx,700187367,CIRLENE DE SERPA BASTOS,NaN,NaN,NaN,(85) 98824-6484,NaN,cirlene.bastos22@gmail.com
2,2 unidade Fortaleza.xlsx,700191362,NaN,NaN,NaN,NaN,NaN,NaN,limavera864@gmail.com
3,2 unidade Fortaleza.xlsx,700187547,NaN,NaN,NaN,NaN,NaN,NaN,sarahjannysil26@gmail.com
4,2 unidade Fortaleza.xlsx,700190977,NaN,NaN,NaN,NaN,NaN,NaN,rejannesilva48@gmail.com


In [13]:
dados_excel = dados_excel.rename(columns={'E-mail': 'email'})

In [14]:
dados_excel

,Nome da Origem,Record ID,Nome,Sobrenome,First name,Número de celular,Número de telefone,Número de telefone do WhatsApp,email
0,2 unidade Fortaleza.xlsx,732319301,Raquel,Felix,NaN,NaN,NaN,NaN,raquelfber@gmail.com
1,2 unidade Fortaleza.xlsx,700187367,CIRLENE DE SERPA BASTOS,NaN,NaN,NaN,(85) 98824-6484,NaN,cirlene.bastos22@gmail.com
2,2 unidade Fortaleza.xlsx,700191362,NaN,NaN,NaN,NaN,NaN,NaN,limavera864@gmail.com
3,2 unidade Fortaleza.xlsx,700187547,NaN,NaN,NaN,NaN,NaN,NaN,sarahjannysil26@gmail.com
4,2 unidade Fortaleza.xlsx,700190977,NaN,NaN,NaN,NaN,NaN,NaN,rejannesilva48@gmail.com
...,...,...,...,...,...,...,...,...,...
12498,2 unidade Piracicaba.xlsx,53988049,JULIANA APARECIDA RODRIGUES DA SILVA,Aparecida Rodrigues Da Silva,JULIANA,5.519974e+12,"+555,51997E+12",NaN,julianarodrigues2y@gmail.com
12499,2 unidade Piracicaba.xlsx,53947747,CYNTHYA STEPHANYE CARDOSO DE BRITO,STEPHANYE CARDOSO DE BRITO,CYNTHYA,5.519982e+12,+5519981792074,NaN,cynthya2.cardoso@gmail.com
12500,2 unidade Piracicaba.xlsx,53947557,CONCEIÇÃO,APARECIDA AUGUSTO DE JESUS,CONCEIÇÃO,5.519992e+12,+5519991625805,5.519992e+12,aha_achouceissa@hotmail.com
12501,2 unidade Piracicaba.xlsx,521663541,Vitoria,Silva Lima,VITORIA,5.519998e+12,+551933744050,NaN,viihlima814@gmail.com


In [22]:
#Faço a query no DBeaver, buscando os desfiliados na tabela de filiados. Baixo um CSV, e faço e salvo no Ubuntu e faço a leitura desse csv via chunks, particionando os dados.
caminho_arquivo_csv = '/home/gabriel/dbt-projects/python_files/analises/files/stg_pacientes_202309200944.csv'

tamanho_do_chunk = 10000

chunks = pd.read_csv(caminho_arquivo_csv, chunksize=tamanho_do_chunk)

dataframes_mesclados = []

In [32]:
# Faz a interação dentro dos chunks
for chunk in chunks:

    chunk_merge = pd.merge(dados_excel, chunk, on='email', how='inner')
    
    dataframes_mesclados.append(chunk_merge)

# Concatene os DataFrames mesclados em um único DataFrame final
df_final = pd.concat(dataframes_mesclados)

In [33]:
colunas_desejadas = [
    'Nome da Origem', 'Record ID', 'Nome', 'Sobrenome', 'First name',
    'Número de celular', 'Número de telefone', 'Número de telefone do WhatsApp',
    'email', 'celular'
]

df_final = df_final[colunas_desejadas]


In [34]:
df_final

,Nome da Origem,Record ID,Nome,Sobrenome,First name,Número de celular,Número de telefone,Número de telefone do WhatsApp,email,celular
0,2 unidade Fortaleza.xlsx,700188921,Isabelle Ferreira Rodrigues,NaN,NaN,NaN,(85) 99706-1526,NaN,isabellefrodrigues3@gmail.com,(85) 99706-1526
1,2 unidade Fortaleza.xlsx,587578167,Irmilene,Da Costa Aquino,IRMILENE,5.585994e+12,+5585994373830,NaN,milene.costacc@hotmail.com,(85) 99437-3830
2,2 unidade Fortaleza.xlsx,327574323,Edilene Camelo De Souza,NaN,NaN,NaN,NaN,NaN,edilenecsmelo@gmail.com,(85) 99946-9198
3,2 unidade Fortaleza.xlsx,143898709,OZAIAS,DA SILVA RODRIGUES,NaN,NaN,+55-27-9927-74252,NaN,clinicaamorsaude@gmail.com,(85) 98403-4245
4,2 unidade Fortaleza.xlsx,143898709,OZAIAS,DA SILVA RODRIGUES,NaN,NaN,+55-27-9927-74252,NaN,clinicaamorsaude@gmail.com,(85) 98507-7422
...,...,...,...,...,...,...,...,...,...,...
2,2 unidade Fortaleza.xlsx,130093047,MARA,CELIA DA SILVA SANTOS,MARA,8.587250e+09,NaN,NaN,maraceara_@hotmail.com,(85) 98480-1001
3,2 unidade Palhoça.xlsx,526039203,MARIELEN,MACHADO MARTINS,MARIELEN,4.898846e+10,+55(48)98846-0340,NaN,marielenmartins271@gmail.com,(48) 98846-0340
4,2 unidade Palhoça.xlsx,201451393,CLOTILDES,OLIVIA LEITE,NaN,NaN,+5548984113722,NaN,clotildesoleite@gmail.com,(48) 98411-3722
5,2 unidade Palhoça.xlsx,147144066,PÂMELA,LEITZKE PEREIRA,PÂMELA,5.548985e+12,+5548985047010,NaN,leitzkep@gmail.com,(48) 98466-6986


In [35]:
from openpyxl import Workbook

In [36]:
nome_do_arquivo = 'celcrm.xlsx'
df_final.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel